In [39]:
import pandas as pd 
import numpy as np 
import yfinance as yf
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta


methodology -  https://www.cmegroup.com/education/demos-and-tutorials/fed-funds-futures-probability-tree-calculator.html


futures api - https://blog.quantinsti.com/download-futures-data-yahoo-finance-library-python/

In [56]:
#parsin fed funds futures prices 
#https://blog.quantinsti.com/download-futures-data-yahoo-finance-library-python/
futures_data = yf.download("ZQ=F", start="2000-01-01", end="2022-11-09")
futures_data.tail(3)

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-11-04,96.214996,96.217499,96.212502,96.214996,96.214996,40467
2022-11-07,96.214996,96.217499,96.214996,96.214996,96.214996,7539
2022-11-08,96.214996,96.217499,96.214996,96.217499,96.217499,7187


In [57]:
#data preparation fed funds futures 
del futures_data['Low']
del futures_data['High']
del futures_data['Adj Close']
del futures_data['Volume']
del futures_data['Open']


futures_data.reset_index(inplace=True)
futures_data.columns = ['date', 'price']
#futures_data['date'] = futures_data['date'].apply(lambda x: datetime.strftime(x, "%Y-%m-%d"))

fdf = futures_data.copy()
fdf.head()

,date,price
0,2000-09-01,93.474998
1,2000-09-05,93.474998
2,2000-09-06,93.474998
3,2000-09-07,93.474998
4,2000-09-08,93.474998


In [63]:
#parsing dates and rate boundaries set by FOMC 
url = 'https://en.wikipedia.org/wiki/History_of_Federal_Open_Market_Committee_actions'
html = requests.get(url).content
df_list = pd.read_html(html)
df = df_list[1]

In [64]:
#function to get avg rate, instead of interval values

def split_rate(s):
    if chr(8211) in s:
        wow = s.split(chr(8211))
        s1 = wow[0][:-1]
        s2 = wow[1][:-1]
        return (float(s1) + float(s2)) / 2
    else:
        return float(s[:-1]) 


In [65]:
#data prep and clean 
df['dt'] = df['Date'].apply(lambda x:datetime.strptime(x, '%B %d, %Y'))
df['dow'] = df['dt'].apply(lambda x : datetime.weekday(x))

#df['dt'] = df['dt'].apply(lambda x: datetime.strftime(x, "%Y-%m-%d"))

del df ['Notes']
del df['Date']
del df['Discount Rate']
del df['Votes']
del df["Unnamed: 5"]

df.columns = ['ffr', 'date', 'dow']

df['r'] = df['ffr'].apply(split_rate)
del df['ffr']


df.head()

,date,dow,r
0,2022-11-02,2,3.875
1,2022-09-21,2,3.125
2,2022-07-27,2,2.375
3,2022-06-15,2,1.625
4,2022-05-04,2,0.875


In [66]:
#adding day before and day after 
def N_filter(dt, dow):
       
        if dow >= 4: 
            return dt + timedelta(days=(7-dow))
        else: 
            return dt + timedelta(days=1)

def B_filter(dt, dow):
       
        if dow == 6: 
            return dt - timedelta(days=(2))
        elif dow == 0:
            return dt - timedelta(days=3) 
        else: 
            return dt - timedelta(days=1)


df['NextDay'] = df.apply(lambda x: N_filter(x['date'], x['dow']), axis = 1)
df['PrevDay'] = df.apply(lambda x: B_filter(x['date'], x['dow']), axis = 1)

In [67]:
df

,date,dow,r,NextDay,PrevDay
0,2022-11-02,2,3.875,2022-11-03,2022-11-01
1,2022-09-21,2,3.125,2022-09-22,2022-09-20
2,2022-07-27,2,2.375,2022-07-28,2022-07-26
3,2022-06-15,2,1.625,2022-06-16,2022-06-14
4,2022-05-04,2,0.875,2022-05-05,2022-05-03
...,...,...,...,...,...
102,2000-08-22,1,6.500,2000-08-23,2000-08-21
103,2000-06-28,2,6.500,2000-06-29,2000-06-27
104,2000-05-16,1,6.500,2000-05-17,2000-05-15
105,2000-03-21,1,6.000,2000-03-22,2000-03-20


In [71]:
#creating a dictionary with futures prices 

fdf['date'] = fdf['date'].apply(lambda x: datetime.strftime(x, "%Y-%m-%d"))

priceoffutures_dict = {}
for i in range(fdf.shape[0]):
    priceoffutures_dict[fdf['date'][i]] = fdf['price'][i]

In [80]:
def rowinpriceoffutures(i):
    for j in range(len(fdf['date'])):
        if fdf['date'][j] > i:
            return j-1
    return -1

In [81]:
df['rowinpriceoffutures'] = df['date'].apply(lambda x : rowinpriceoffutures(x))

In [88]:
test_df = df.copy()
#test_fdf['date'] = test_fdf['date'].apply(lambda x: datetime.strftime(x, "%Y-%m-%d"))

test_df['date'] = test_df['date'].apply(lambda x: datetime.strftime(x, "%Y-%m-%d"))
test_df['NextDay'] = test_df['NextDay'].apply(lambda x: datetime.strftime(x, "%Y-%m-%d"))
test_df['PrevDay'] = test_df['PrevDay'].apply(lambda x: datetime.strftime(x, "%Y-%m-%d"))



0      2022-11-02
1      2022-09-21
2      2022-07-27
3      2022-06-15
4      2022-05-04
          ...    
102    2000-08-22
103    2000-06-28
104    2000-05-16
105    2000-03-21
106    2000-02-02
Name: date, Length: 107, dtype: object

In [93]:
test_df['Rate'] = test_df['date'].apply(lambda x : priceoffutures_dict[x])

KeyError: '2020-03-15'

In [96]:
priceoffutures_dict['2020-03-13']

99.27999877929688

In [27]:
t_m['Ratedaybefore'] = t_m['bd'].apply(lambda x : priceoffutures_dict[x])

KeyError: Timestamp('2008-01-21 00:00:00')

In [ ]:
priceoffutures_dict = {}
for i in range(priceoffutures.shape[0]):
    priceoffutures_dict[priceoffutures['Date'][i]] = priceoffutures[' value'][i]

In [20]:
t_f

,dt,p,rowinpriceoffutures
0,2000-09-01,93.474998,0
1,2000-09-05,93.474998,1
2,2000-09-06,93.474998,2
3,2000-09-07,93.474998,3
4,2000-09-08,93.474998,4
...,...,...,...
5549,2022-11-02,96.217499,5549
5550,2022-11-03,96.214996,5550
5551,2022-11-04,96.214996,5551
5552,2022-11-07,96.214996,5552


In [203]:
tdf['p'] = tdf['dt'].apply(lambda x: priceoffutures_dict[x])
#tdf['_p'] = tdf['bd'].apply(lambda x: priceoffutures_dict[x])
#tdf['p_'] = tdf['nd'].apply(lambda x: priceoffutures_dict[x])



KeyError: Timestamp('2020-03-15 00:00:00')